# Class II - An introduction to JuMP

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of JuMP. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) first.

**Warning! This notebook is an introduction to JuMP v0.18. However, JuMP is currently undergoing a re-write. In the near future, some aspects of JuMP will change. But don't worry, the majority of this tutorial is still relevant in the future.**

This tutorial doesn't exist in isolation. Some other good resources for learning JuMP are
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.18/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)

Before we start, run the following magic sauce to install the required Julia packages and check that we're good to go.

In [69]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `C:\Users\Oscar\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `C:\Users\Oscar\.julia\registries\JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
Excellent! Everything is good to go!


## The basics

First, load the JuMP package into your current environment.

In [70]:
using JuMP

Start building a JuMP model like so,

In [71]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

Feasibility problem with:
 * 0 linear constraints
 * 3 variables
Solver is default solver

We can also create arrays of JuMP variables.

In [72]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

4-element Array{Variable,1}:
 x[1]
 x[2]
 x[3]
 x[4]

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [73]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
x

x[i,j] >= .. for all i in {1,2}, j in {A,B}

We can also add a condition to filter out some of the variables

In [74]:
model = Model()
@variable(model, x[i = 1:4, j = [:A, :B]; isodd(i)] >= i)
x

x[i,j] >= .. for all i in {1,2,3,4}, j in {A,B} s.t. isodd(i)

What if I want to add two variables with the same name?

In [75]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

x

┌ Warning: A variable or constraint named x is already attached to this model. If creating variables programmatically, use the anonymous variable syntax x = @variable(m, [1:N], ...).
└ @ JuMP C:\Users\Oscar\.julia\packages\JuMP\PbnIJ\src\JuMP.jl:852


### Quiz Question

What is the value of the following?

In [76]:
JuMP.getlowerbound(x)

2.0

### Anonymous variables

So far, we've only added *named* variables. This works great until we want to add two variables with the same name! To work around this isse, we can add *anonymous* variables.

In [77]:
model = Model()
x = @variable(model)
@show JuMP.getlowerbound(x)
x = @variable(model, lowerbound = 1)
@show JuMP.getlowerbound(x)
x = @variable(model, [i = 1:2], lowerbound = i)
@show JuMP.getlowerbound(x)

model[:x]  # This throws an error

JuMP.getlowerbound(x) = -Inf
JuMP.getlowerbound(x) = 1.0
JuMP.getlowerbound(x) = [1.0, 2.0]


KeyError: KeyError: key "No object with name x" not found

So what's the difference between the *named* and *anonymous* syntax? 

Well, `@variable(model, x >= 2)` is (roughly) equivalent to

In [78]:
model = Model()
x = model[:x] = @variable(model, lowerbound = 2, basename = "x")
model

Feasibility problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

In [79]:
model = Model()
@variable(model, x >= 2)
model

Feasibility problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

Although uneeded for this tutorial, we can also create binary and integer variables as follows:

In [80]:
model = Model()
@variable(model, x >= 1, Int)
@variable(model, y, Bin)
model

Feasibility problem with:
 * 0 linear constraints
 * 2 variables: 1 binary, 1 integer
Solver is default solver

### Constraints

Now that we've seen how to create variables, let's look at constraints. Much of the syntax should be familiar.

In [81]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

Feasibility problem with:
 * 3 linear constraints
 * 2 variables
Solver is default solver

In [82]:
model[:c_equal_to]

2 x + y == 1

In [83]:
anonymous_constraint = @constraint(model, [i = 1:2], i * x <= y)

model

Feasibility problem with:
 * 5 linear constraints
 * 2 variables
Solver is default solver

### Objective Functions

Now let's look at the objective function.

In [84]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

Minimization problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

In [85]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

Maximization problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

### Solving a Model

Once we've formulated a model, the next step is to solve it. This requires a solver.

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/installation.html#getting-solvers) contains a list of the supported solvers and the types of problems each solver supports.

We're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPKMathProgInterface.jl](https://github.com/JuliaOpt/GLPKMathProgInterface.jl) package.

In [86]:
using GLPKMathProgInterface
# Define an instance of the GLPK solver.
const GLPK_SOLVER = GLPKMathProgInterface.GLPKSolverLP()

GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP(false, :Simplex, Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}())

The second solver is the COIN-OR [Interior Point OPTimizer(Ipopt)](https://projects.coin-or.org/Ipopt). This solver supports nonlinear programs with continous variables.

Ipopt is available via the [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package.

In [87]:
using Ipopt
const IPOPT_SOLVER = IpoptSolver(print_level=0)

IpoptSolver(Tuple[(:print_level, 0)])

There are two ways to add a solver to a JuMP model:

In [88]:
model = Model()
JuMP.setsolver(model, GLPK_SOLVER)

# ... or ...

model = Model(solver = IPOPT_SOLVER)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Ipopt

If you try to solve an unsupported problem type, an error will be thrown:

In [89]:
model = Model(solver = GLPK_SOLVER)
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2)
solve(model)

MethodError: MethodError: no method matching NonlinearModel(::GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP)
Closest candidates are:
  NonlinearModel(!Matched::IpoptSolver) at C:\Users\Oscar\.julia\packages\Ipopt\ZuxdU\src\MPBWrapper.jl:38
  NonlinearModel(!Matched::Int64) at C:\Users\Oscar\.julia\packages\MathProgBase\rVlFR\src\SolverInterface\SolverInterface.jl:29
  NonlinearModel() at C:\Users\Oscar\.julia\packages\MathProgBase\rVlFR\src\SolverInterface\SolverInterface.jl:27

That error isn't very nice, but don't worry, the new version of JuMP has much nicer errors. This sort of thing shouldn't happen in the future.

In [90]:
JuMP.setsolver(model, IPOPT_SOLVER)
status = solve(model)

println("The solution status is: $(status)")

The solution status is: Optimal


In [91]:
x_value = JuMP.getvalue(x)
obj_value = JuMP.getobjectivevalue(model)

println("      x | $(x_value)")
println("    π/2 | $(π/2)")
println("--------+----------------------")
println("cos²(x) | $(obj_value)")

      x | 1.5707963267948966
    π/2 | 1.5707963267948966
--------+----------------------
cos²(x) | 3.749399456654644e-33


## Example: Simple Economic Dispatch

*This example is adapted from a [tutorial given at the 2015 Grid Science Winter School](https://github.com/JuliaOpt/juliaopt-notebooks/blob/3110eddaf5effdecfee687739295bea05731ba33/notebooks/Dvorkin%20-%20Power%20systems%20-%20Economic%20dispatch%20and%20Unit%20commitment.ipynb).*

Economic dispatch (ED) is an optimization problem that minimizes the cost of supplying energy demand subject to operational constraints on power system assets. In its simplest instantiation, ED is a Linear Programing (LP) problem solved for an aggregated load and wind forecast and for a single infinitesimal moment. Mathematically, the ED problem can be written as follows:
$$
\min \sum_{i \in I} c^g_{i} \cdot g_{i} + c^w \cdot w,
$$
where $c_{i}$ and $g_{i}$ are the incremental cost ($\$/MWh$) and power output ($MW$) of the $i^{th}$ thermal generator, respectively, and $c^w$ and $w$ are the incremental cost ($\$/MWh$) and wind power injection ($MW$), respectively.

s.t.

<li> Minimum ($g^{\min}$) and maximum ($g^{\max}$) limits on power outputs of the thermal generators: </li>
$$
g^{\min}_{i} \leq g_{i} \leq g^{\max}_{i}.
$$
<li>Constraint on the wind power injection:</li>
$$
0 \leq w \leq w^f, 
$$
where $w$ and $w^f$ are the wind power injection and wind power forecast, respectively.

<li>Power balance constraint:</li>
$$
\sum_{i \in I} g_{i} + w = d, 
$$
where $d$ is the demand.

Further reading on ED models can be found in A. J. Wood, B. F. Wollenberg, and G. B. Sheblé, "Power Generation, Operation and Control", Wiley, 2013.

### JuMP Implementation of Economic Dispatch 

First, we need to load some plotting packages.  Note, complitaton may take a few minutes.

In [92]:
using Interact
using Plots

Now, we define some problem data for a model with two thermal generators and one wind generator.

In [93]:
# Define some input data about the test system
# Maximum power output of generators
const GENERATION_MAX = [1000,1000]
# Minimum power output of generators
const GENERATION_MIN = [0,300]
# Incremental cost of generators 
const COST_GENERATION = [50,100]
# Incremental cost of wind generators
const COST_WIND = 50
# Total demand
const DEMAND = 1500
# Wind forecast
const WIND_MAX = 200;

In the next cell, we create a Julia function that formulates and solves the economic dispatch problem.

In [94]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)

    economic_dispatch = Model(solver = GLPK_SOLVER) 
    
    # Define decision variables    
    @variables(economic_dispatch, begin
        g[i=1:2]  # Thermal generation (MW).
        w >= 0  # Wind power (MW).
    end)

    # Define the objective function
    @objective(economic_dispatch, Min, 
        sum(cost_of_thermal[i] * g[i] for i in 1:2) + cost_of_wind * w
    )

    # Define the constraint on the maximum and minimum power output of each generator.
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end
    
    @constraints(economic_dispatch, begin
        # Define the constraint on the wind power injection
        w <= WIND_MAX
        # Define the power balance constraint
        sum(g[i] for i in 1:2) + w == DEMAND
    end)

    # Solve statement
    solve(economic_dispatch)
    
    # Return the optimal value of the objective function and its minimizers
    # as a NamedTuple.
    return (
        generation = getvalue(g), 
        wind_generation = getvalue(w),
        wind_spillage = WIND_MAX - getvalue(w),
        cost = getobjectivevalue(economic_dispatch)
    )
end

# Solve the economic dispatch problem
solution = solve_economic_dispatch()

println("Dispatch")
println("   Generators: ", solution.generation, " MW")
println("         Wind: ", solution.wind_generation, " MW")
println("Wind spillage: ", solution.wind_spillage, " MW") 
println("----------------------------------")
println("Total cost: \$", solution.cost)  

Dispatch
   Generators: [1000.0, 300.0] MW
         Wind: 200.0 MW
Wind spillage: 0.0 MW
----------------------------------
Total cost: $90000.0


### Economic dispatch with adjustable incremental costs

In the following exercise we introduce a manipulator to vary the cost of wind generation and observe its impact the total cost, dispatch of generators G1 and G2, utilization of available wind under different values of the incremental cost of generator G1.

In [95]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3.5)
    # Define the vectors of outputs
    objectives = Float64[] 
    wind_dispatch = Float64[]
    G1_dispatch = Float64[]
    G2_dispatch = Float64[]
    
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.01:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        # Add the solution of the economic dispatch problem to the respective vectors
        push!(objectives, solution.cost)
        push!(wind_dispatch, solution.wind_generation)
        push!(G1_dispatch, solution.generation[1])
        push!(G2_dispatch, solution.generation[2])
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, objectives,
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, G1_dispatch,
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, G2_dispatch,
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, wind_dispatch,
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:cost_of_wind=>Widget{:slider,Float64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:index=>Observable{Any} with 2 listeners. Value:
13,:formatted_vals=>Observable{Any} with 1 listeners. Value:
["50.0", "55.0", "60.0", "65.0", "70.0", "75.0", "80.0", "85.0", "90.0", "95.0"  …  "130.0", "135.0", "140.0", "145.0", "150.0", "155.0", "160.0", "165.0", "170.0", "175.0"],:formatted_value=>Observable{String} with 1 listeners. Value:
"110.0",:value=>Observable{Float64} with 2 listeners. Value:
110.0), Observable{Float64} with 2 listeners. Value:
110.0, Scope("knockout-component-a20e3c4e-598b-467c-8d93-f7be3dd51d2d", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>26,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("formatted_vals"=>(Observable{Any} with 1 listeners. Value:
["50.0", "55.0", "60.0", "65.0", "70.0", "75.0", "80.0", "85.0", "90.0", "95.0"  …  "130.0", "135.0", "140.0", "145.0", "150.0", "155.0", "160.0", "165.0", "170.0", "175.0"], nothing),"changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"formatted_value"=>(Observable{String} with 1 listeners. Value:
"110.0", nothing),"index"=>(Observable{Any} with 2 listeners. Value:
13, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\Q4IkI\\src\\..\\assets\\all.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\Q4IkI\\src\\..\\assets\\style.css", "C:\\Users\\Oscar\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("formatted_vals"=>Any[JSString("(function (val){return (val!=this.model[\"formatted_vals\"]()) ? (this.valueFromJulia[\"formatted_vals\"]=true, this.model[\"formatted_vals\"](val)) : undefined})")],"_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueU

## Nonlinear example

JuMP can also be used to solve non-linear problems (NLP). We saw a brief hint of this earlier when we used Ipopt and the `@NLobjective` macro. In the next example, we add nonlinearity to the cost of the second generator. The new cost of generation is
$$\text{generation_cost} = c_1^g\times g_1 + \frac{c_2^g\times  g_2^{1.5}}{\sqrt{1000}}.$$


In [98]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_nonlinear_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)
    economic_dispatch = Model(solver = IPOPT_SOLVER) 
    
    @variables(economic_dispatch, begin
        g[i=1:2] >= 0
        w >= 0
    end)

    # ===========================================================
    # Look! This bit changed.
    function generator_cost(g1, g2)
        return cost_of_thermal[1] * g1 + cost_of_thermal[2] * g2^1.5 / sqrt(1000)
    end
    JuMP.register(economic_dispatch, :generator_cost, 2, generator_cost, autodiff=true)
    @NLobjective(economic_dispatch, Min, generator_cost(g[1], g[2]) + cost_of_wind * w)
    # ===========================================================
    
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end    
    @constraints(economic_dispatch, begin
        w <= WIND_MAX
        sum(g[i] for i in 1:2) + w == DEMAND
    end)
    solve(economic_dispatch)
    return (
        generation = getvalue(g), 
        wind_generation = getvalue(w),
        wind_spillage = WIND_MAX - getvalue(w),
        cost = getobjectivevalue(economic_dispatch)
    )
end

solve_nonlinear_economic_dispatch

In [99]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3)
    # Define the vectors of outputs
    objectives = Float64[] 
    wind_dispatch = Float64[]
    G1_dispatch = Float64[]
    G2_dispatch = Float64[]
    
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.1:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_nonlinear_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        # Add the solution of the economic dispatch problem to the respective vectors
        push!(objectives, solution.cost)
        push!(wind_dispatch, solution.wind_generation)
        push!(G1_dispatch, solution.generation[1])
        push!(G2_dispatch, solution.generation[2])
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, objectives,
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, G1_dispatch,
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, G2_dispatch,
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, wind_dispatch,
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Widget{:manipulate,Any}(OrderedDict{Symbol,Any}(:cost_of_wind=>Widget{:slider,Float64}(OrderedDict{Symbol,Any}(:changes=>Observable{Int64} with 1 listeners. Value:
0,:index=>Observable{Any} with 2 listeners. Value:
11,:formatted_vals=>Observable{Any} with 1 listeners. Value:
["50.0", "55.0", "60.0", "65.0", "70.0", "75.0", "80.0", "85.0", "90.0", "95.0"  …  "105.0", "110.0", "115.0", "120.0", "125.0", "130.0", "135.0", "140.0", "145.0", "150.0"],:formatted_value=>Observable{String} with 1 listeners. Value:
"100.0",:value=>Observable{Float64} with 2 listeners. Value:
100.0), Observable{Float64} with 2 listeners. Value:
100.0, Scope("knockout-component-f67fed6a-0f0c-45ca-91f7-cef1fff56aa8", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>21,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("formatted_vals"=>(Observable{Any} with 1 listeners. Value:
["50.0", "55.0", "60.0", "65.0", "70.0", "75.0", "80.0", "85.0", "90.0", "95.0"  …  "105.0", "110.0", "115.0", "120.0", "125.0", "130.0", "135.0", "140.0", "145.0", "150.0"], nothing),"changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"formatted_value"=>(Observable{String} with 1 listeners. Value:
"100.0", nothing),"index"=>(Observable{Any} with 2 listeners. Value:
11, nothing)), Set(String[]), nothing, Any["knockout"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout.js", "knockout_punches"=>"C:\\Users\\Oscar\\.julia\\packages\\Knockout\\JIqpG\\src\\..\\assets\\knockout_punches.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\Q4IkI\\src\\..\\assets\\all.js", "C:\\Users\\Oscar\\.julia\\packages\\InteractBase\\Q4IkI\\src\\..\\assets\\style.css", "C:\\Users\\Oscar\\.julia\\packages\\InteractBulma\\Ohu5Y\\src\\..\\assets\\main.css"], Dict{Any,Any}("formatted_vals"=>Any[JSString("(function (val){return (val!=this.model[\"formatted_vals\"]()) ? (this.valueFromJulia[\"formatted_vals\"]=true, this.model[\"formatted_vals\"](val)) : undefined})")],"_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueU